In [1]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.datasets import make_blobs
# Set the random seed for reproducibility
random_seed = 42

# Generate blob data
X, y = make_blobs(n_samples=100, centers=2, random_state=random_seed)

# Create pipelines 
# 1. Using make_pipeline function
# 2. Using Pipeline class
pipeline_maked = make_pipeline(MinMaxScaler(), StandardScaler())
pipeline_with_class = Pipeline([["min-max", MinMaxScaler()], ["std-scaler", StandardScaler()]])

# Fit and transform the data
r_maked = pipeline_maked.fit_transform(X, y)
r_with_class = pipeline_with_class.fit_transform(X, y)

# Compare, assrting equality
assert np.allclose(r_maked, r_with_class)
print("Pipeline results are equal")

Pipeline results are equal


In [2]:
## The Minerva model
model_str = """
class_path: minerva.models.nets.classic_ml_pipeline.ClassicMLModel
init_args:
    backbone:
        class_path: torch.nn.Identity
        init_args: {}
    head: 
        class_path: minerva.models.nets.classic_ml_pipeline.SklearnPipeline
        init_args:
            steps:
                -   - std-scaler
                    - class_path: sklearn.preprocessing.StandardScaler
                -   - logreg
                    - class_path: sklearn.linear_model.LogisticRegression 
    use_only_train_data: True
"""

In [3]:
## The DAGHAR Kuhar data module
daghar_kuhar_data_module = """
class_path: minerva.data.data_modules.har.MultiModalHARSeriesDataModule
init_args:
  data_path: "/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/KuHar/"
  feature_prefixes: ["accel-x", "accel-y", "accel-z", "gyro-x", "gyro-y", "gyro-z"]
  label: "standard activity code"
  features_as_channels: True
  cast_to: "float32"
  batch_size: 64
"""

In [4]:
## The Trainer

trainer_str = """
class_path: lightning.Trainer
init_args:
  # Accelerator to use (CPU, GPU, TPU, ..).
  accelerator: cpu
  # Number of accelerator devices to use.
  devices: 1
  # Strategy to use for distributed training.
  strategy: auto
  # Maximum number of epochs to train.
  max_epochs: 1
  # Logger and callback configuration (load from file)
  deterministic: warn
  
  
  logger: null
  callbacks: null
  enable_checkpointing: False
"""

In [5]:
import tempfile

tmp_dir = tempfile.TemporaryDirectory()
model_file = tempfile.NamedTemporaryFile(dir=tmp_dir.name, suffix=".yaml", delete=False)
with open(model_file.name, "w") as f:
    f.write(model_str)
    print(f"Model file: {model_file.name}")
        
data_module_file = tempfile.NamedTemporaryFile(dir=tmp_dir.name, suffix=".yaml", delete=False)
with open(data_module_file.name, "w") as f:
    f.write(daghar_kuhar_data_module)
    print(f"Data module file: {data_module_file.name}")
    
trainer_file = tempfile.NamedTemporaryFile(dir=tmp_dir.name, suffix=".yaml", delete=False)
with open(trainer_file.name, "w") as f:
    f.write(trainer_str)  
    print(f"Trainer file: {trainer_file.name}")
   
   
output_file = tempfile.NamedTemporaryFile(dir=tmp_dir.name, suffix=".yaml", delete=False)

Model file: /tmp/tmp8b4jw7j9/tmpv75zqxao.yaml
Data module file: /tmp/tmp8b4jw7j9/tmpcnwxbfrc.yaml
Trainer file: /tmp/tmp8b4jw7j9/tmpgbiqstbn.yaml


In [6]:
cmd_string = f"python -W ignore -m minerva.pipelines.lightning_pipeline --print_config  --model {model_file.name} --trainer {trainer_file.name} run --data {data_module_file.name} "
print(f"Running command: {cmd_string}\n")

output = !$cmd_string

with open(output_file.name, "w") as f:
    f.write("\n".join(output))
    print(f"Output file: {output_file.name}")

Running command: python -W ignore -m minerva.pipelines.lightning_pipeline --print_config  --model /tmp/tmp8b4jw7j9/tmpv75zqxao.yaml --trainer /tmp/tmp8b4jw7j9/tmpgbiqstbn.yaml run --data /tmp/tmp8b4jw7j9/tmpcnwxbfrc.yaml 

Output file: /tmp/tmp8b4jw7j9/tmpl8i_5rsx.yaml


In [7]:
!cat {output_file.name} 2>/dev/null

model:
  class_path: minerva.models.nets.classic_ml_pipeline.ClassicMLModel
  init_args:
    backbone:
      class_path: torch.nn.Identity
    head:
      class_path: minerva.models.nets.classic_ml_pipeline.SklearnPipeline
      init_args:
        steps:
        - - std-scaler
          - 'Unable to serialize instance {''class_path'': ''sklearn.preprocessing.StandardScaler''}'
        - - logreg
          - 'Unable to serialize instance {''class_path'': ''sklearn.linear_model.LogisticRegression''}'
        memory: null
        verbose: false
    use_only_train_data: true
    test_metrics: null
    sklearn_model_save_path: null
    flatten: true
    adapter: null
trainer:
  class_path: lightning.Trainer
  init_args:
    accelerator: cpu
    strategy: auto
    devices: 1
    num_nodes: 1
    precision: null
    logger: null
    callbacks: null
    fast_dev_run: false
    max_epochs: 1
    min_epochs: null
    max_steps: -1
    min_steps: null
    max_time: null
    limit_train_batches: n

In [8]:
! python -m minerva.pipelines.lightning_pipeline --config {output_file.name} run --task fit

/home/vscode/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 0
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/workspace/lightning_logs/version_1

  | Name     | Type     | Params | Mode
---------------------------------------------
0 | backbone | Identity | 0      | eval
---------------------------------------------
0         Trainable params
0         Non-trainable params
0         Total params
0.000     Total estimated model params size (MB)
0         Modules in train mode
1         Mo

In [9]:
tmp_dir.cleanup()

In [38]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
import torch
from torch.utils.data import DataLoader
import lightning as L
from minerva.models.nets.classic_ml_pipeline import ClassicMLModel, SklearnPipeline
from minerva.utils.data import SimpleDataset

def test_sklearn_pipeline():
    X, y = make_blobs(n_samples=64, centers=2, random_state=42)
    train_dataset = SimpleDataset(X, y)
    train_dataloader = DataLoader(train_dataset, batch_size=64)
    pipeline = ClassicMLModel(
        backbone = torch.nn.Identity(),
        head = SklearnPipeline(
            [["min-max", MinMaxScaler()], ["log-reg", LogisticRegression(random_state=42, max_iter=5)]]
        )
    )
    trainer = L.Trainer(fast_dev_run=True)
    trainer.fit(pipeline, train_dataloader)
    
test_sklearn_pipeline()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type     | Params | Mode
---------------------------------------------
0 | backbone | Identity | 0      | eval
---------------------------------------------
0         Trainable params
0         Non-trainable params
0         Total params
0.000     Total estimated model params size (MB)
0         Modules in train mode
1         Modules in eval mode


Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 70.21it/s] 

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 54.34it/s]
